In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from collections import Counter
from fastFM import sgd
import lightgbm as lgb
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import tqdm
import gc
import os
import sys
sys.path.append('../code/utils')
import data_utils as du
import perf_utils as pu

In [2]:
df_train = du.load_raw_data("train")
df_test = du.load_raw_data("test")
df_ad = du.load_raw_data("ad")

In [3]:
user_one_feat_names = ['age', 'gender', 'education', 'consumptionAbility', 'LBS',
                  'carrier', 'house']  # one user has only one value
user_multi_feat_names = ['marriageStatus', 'interest1', 'interest2', 'interest3',
                   'interest4', 'interest5', 'kw1', 'kw2', 'kw3', 'topic1',
                   'topic2', 'topic3', 'appIdInstall', 'appIdAction', 'ct', 'os']  # one user can have more than one value
user_feat_names = user_one_feat_names + user_multi_feat_names

In [4]:
ad_num_feat_names = ['creativeSize']
ad_cat_feat_names = ['advertiserId', 'campaignId', 'creativeId', 'adCategoryId', 'productId', 'productType']
ad_feat_names = ad_num_feat_names + ad_cat_feat_names

In [5]:
# both 'creativeId' and 'aid' are unique to advertise. so you can do nothing actually
# I just remove 'creativeId' and add 'aid' to make my intention clearer
ad_feat_names.remove("creativeId")  # every ad has a unique 'creativeId', so it's useless
ad_feat_names.append("aid")

In [6]:
with pu.profiler("loading and joining user and ad data"):
    X, feat_names = du.quick_join(ad_user=df_train, 
#                                   user_feat_names=user_feat_names, 
                                  user_feat_names=user_one_feat_names,
#                                   ad_feat_names=ad_feat_names
                                  ad_feat_names=ad_num_feat_names)
    gc.collect()
print("Combined Matrix Shape: {}".format(X.shape))
print("Feature Names Count: {}".format(len(feat_names)))
print("Memory usage at this moment :{}".format(pu.get_memory_str()))

loading ad matrices: 100%|██████████| 1/1 [00:03<00:00,  3.09s/it]

[07:57:18] Finish loading and joining user and ad data. △M: +624.23MB. △T: 2.8 minutes.
Combined Matrix Shape: (8798814, 897)
Feature Names Count: 897
Memory usage at this moment :964.36MB


In [7]:
y = df_train['label'].values
assert X.shape[0] == y.shape[0]

In [8]:
X = sparse.csr_matrix(X)  # COO to CSR

In [9]:
n_splits = 10  # use 10 for debug
skf = StratifiedKFold(n_splits=n_splits)
split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [10]:
train_index, valid_index = split_indices[0]

# X_train, y_train = X[train_index, :], y[train_index]
# X_valid, y_valid = X[valid_index, :], y[valid_index]
X_train, y_train = X[valid_index, :], y[valid_index]
X_valid, y_valid = X[train_index, :], y[train_index]

In [20]:
def objective(params):
    for k, v in params.items():
        if k in set(["n_iter", "rank"]):
            params[k] = round(v)
    
    fm = sgd.FMClassification(**params)
    fm.fit(X_train, y_train)
    proba_valid = fm.predict_proba(X_valid)
    fm_params = fm.get_params()
    if np.isnan(proba_valid).sum() == 0:
        auc = metrics.roc_auc_score(y_valid, proba_valid)
        print("[{}] Params: {} => AUC: {:.6f}, Results: {}".format(pu.get_time_str(), params, auc, proba_valid))
        return {
            'params': fm_params,
            'loss': -auc,
            'status': STATUS_OK
            }
    else:
        print("[{}] Params: {} => Failed".format(pu.get_time_str(), params))
        return {
            'params': fm_params,
            'loss': 0,
            'status': STATUS_OK
            }

In [21]:
trials = Trials()
best = fmin(objective,
            space={
                'n_iter': hp.loguniform('n_iter', 0, np.log(10000)),
                'init_stdev': hp.loguniform('init_stdev', np.log(1e-3), 0),
                'rank': hp.loguniform('rank', 0, np.log(1000)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e5)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1e-5), np.log(1e5)),
                'step_size': hp.loguniform('step_size', np.log(1e-2), np.log(5e-1))
            }，
            algo=tpe.suggest,
            max_evals=1000,
            trials=trials)

[08:04:04] Params: {'init_stdev': 0.07105403381575823, 'step_size': 0.3680579747448019, 'l2_reg_w': 0.0041885356492519675, 'rank': 17, 'l2_reg_V': 0.013388892737126526, 'n_iter': 7} => AUC: 0.505914, Results: [0.00462332 0.01121735 0.0517332  ... 0.02592886 0.01453001 0.01088113]
[08:04:22] Params: {'init_stdev': 0.02778810112617942, 'step_size': 0.19285123353127304, 'l2_reg_w': 1.695567302203993, 'rank': 43, 'l2_reg_V': 2.0461225223868222, 'n_iter': 10} => AUC: 0.518283, Results: [0.07186489 0.0914847  0.12446915 ... 0.12155941 0.08389619 0.08049025]
[08:06:29] Params: {'init_stdev': 0.002437588007471658, 'step_size': 0.11487463549540866, 'l2_reg_w': 7.295549065802013, 'rank': 507, 'l2_reg_V': 0.00011823201149630977, 'n_iter': 2} => AUC: 0.520546, Results: [0.34263824 0.3840195  0.36814456 ... 0.38731833 0.36662643 0.36573594]


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[08:06:39] Params: {'init_stdev': 0.03360711953325245, 'step_size': 0.4145583984287946, 'l2_reg_w': 96.46161930037411, 'rank': 22, 'l2_reg_V': 74.38266172560736, 'n_iter': 154} => Failed


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[08:08:17] Params: {'init_stdev': 0.13001172300457475, 'step_size': 0.11194163856924393, 'l2_reg_w': 47350.542571127, 'rank': 394, 'l2_reg_V': 0.010011413543219267, 'n_iter': 289} => Failed


...

/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[18:15:06] Params: {'init_stdev': 0.0010130195529530117, 'step_size': 0.06093911672057654, 'l2_reg_w': 39738.38078791644, 'rank': 280, 'l2_reg_V': 3.4105503225349276e-05, 'n_iter': 1237} => Failed
[18:15:46] Params: {'init_stdev': 0.007389597201360314, 'step_size': 0.11940405252758365, 'l2_reg_w': 1.0020628164925667e-05, 'rank': 118, 'l2_reg_V': 0.00030190437626853085, 'n_iter': 5370} => AUC: 0.536429, Results: [2.45556470e-04 6.50195399e-05 7.71401053e-04 ... 1.20036306e-02
 7.05913400e-03 3.06357315e-03]


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[18:17:46] Params: {'init_stdev': 0.004145812881595208, 'step_size': 0.09637868577269963, 'l2_reg_w': 35.345141953551796, 'rank': 433, 'l2_reg_V': 2.0572865657278406e-05, 'n_iter': 2082} => Failed
[18:18:17] Params: {'init_stdev': 0.001365999597512276, 'step_size': 0.07141134030898169, 'l2_reg_w': 1.6730614319367115e-05, 'rank': 83, 'l2_reg_V': 0.00015770555679484993, 'n_iter': 2754} => AUC: 0.559191, Results: [0.00047295 0.0135976  0.02065992 ... 0.05527403 0.02924705 0.00837817]
[18:18:44] Params: {'init_stdev': 0.0011591139964668875, 'step_size': 0.05253663637303801, 'l2_reg_w': 2.8240906410690554e-05, 'rank': 72, 'l2_reg_V': 1.0955584816998021e-05, 'n_iter': 328} => AUC: 0.498499, Results: [0.00119786 0.00193348 0.00406506 ... 0.00242293 0.00095589 0.00061448]
[18:20:25] Params: {'init_stdev': 0.0020663896054916417, 'step_size': 0.11179307219092342, 'l2_reg_w': 6.971227989875099e-05, 'rank': 342, 'l2_reg_V': 6.439887944520118e-05, 'n_iter': 1585} => AUC: 0.534874, Results: [2.31742

[18:39:58] Params: {'init_stdev': 0.007536014455713216, 'step_size': 0.05449053037761237, 'l2_reg_w': 6.1994643364121e-05, 'rank': 141, 'l2_reg_V': 1.4802012927994585e-05, 'n_iter': 5035} => AUC: 0.571266, Results: [2.21169638e-05 1.17465927e-04 1.60039104e-04 ... 1.96856144e-02
 6.39360139e-03 2.30326308e-04]
[18:40:44] Params: {'init_stdev': 0.0011614855245165021, 'step_size': 0.09451896234075992, 'l2_reg_w': 0.0009172608941607521, 'rank': 111, 'l2_reg_V': 0.00014218760792469737, 'n_iter': 404} => AUC: 0.503870, Results: [0.00014601 0.00045678 0.00073758 ... 0.0004919  0.00011769 0.00035809]


In [38]:
print("Best Attempt:")
print(trials.best_trial['result'])

Best Attempt:
{'loss': -0.5755815658707961, 'params': {'init_stdev': 0.002982592686381534, 'random_state': 123, 'step_size': 0.02435446907525723, 'l2_reg_w': 0.00025196049286158295, 'rank': 65, 'l2_reg_V': 0.00010766323454346008, 'n_iter': 5273, 'l2_reg': None}, 'status': 'ok'}


In [46]:
log_folder = "../log/fm/params_tunning"
log_file = "0501.csv"
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [54]:
rows = []
for result in trials.results:
    row = {'auc': -result['loss']}
    for k, v in result['params'].items():
        row[k] = round(v, 7) if type(v) == float else v
    rows.append(row)
df_log = pd.DataFrame(rows)
df_log = df_log.sort_values("auc", ascending=False)

In [55]:
df_log.to_csv(log_path, index=False)

In [ ]:
# print("[{}] Start training FM with SGD optimizer".format(pu.get_time_str()))
# with pu.profiler("training FM Classifier"):
#     fm = sgd.FMClassification(n_iter=100, init_stdev=0.1, l2_reg_w=0,
#                               l2_reg_V=0, rank=20, step_size=0.1)
#     fm.fit(X_train, y_train)
#     proba_valid = fm.predict_proba(X_valid)
#     print("AUC: {:.6f}".format(metrics.roc_auc_score(y_valid, proba_valid)))

In [ ]:
# On the whole dataset
# problem found in FM: most value are either 1 or 0
# 0.49674888197162625
# 0.486489
# 0.504014  # l2_reg_w=1, l2_reg_V=1 
# 0.542916  # l2_reg_w=1, l2_reg_V=10  (n_iter=1000, rank=5, same below if not specified)
# NaN       # l2_reg_w=1, l2_reg_V=100
# 0.540082  # l2_reg_w=1, l2_reg_V=30
# 0.539159  # l2_reg_w=1, l2_reg_V=20
# 0.537865  # l2_reg_w=1, l2_reg_V=5
# 0.510271  # l2_reg_w=1, l2_reg_V=10, n_iter=10000
# 0.498640  # l2_reg_w=1, l2_reg_V=10, n_iter=100
# 0.512793  # l2_reg_w=1, l2_reg_V=10, rank=10
# 0.497214  # l2_reg_w=1, l2_reg_V=30, rank=10
# 0.509446  # l2_reg_w=1, l2_reg_V=5, rank=10
# 0.497834  # l2_reg_w=10, l2_reg_V=1, rank=5
# 0.491871  # l2_reg_w=10, l2_reg_V=1, rank=5
# 0.524053  # l2_reg_w=10, l2_reg_V=0.1, rank=20
# 0.506883  # l2_reg_w=10, l2_reg_V=0.01, rank=20
# 0.502137  # l2_reg_w=0, l2_reg_V=0, rank=20